In [52]:
import time
import sqlite3
import requests
import datetime

import pandas as pd
import shioaji as sj
import re
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar

import mplfinance as mpf

from FuncBase import *

輸入成功!


In [35]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji()
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


<SecurityType.Index: 'IND'> fetch done
<SecurityType.Future: 'FUT'> fetch done


In [36]:
import time
def get_ticks(connection, api, date, codes = str, is_Futures = False):

    dates = pd.to_datetime(date)
    tw_calendar = get_calendar('XTAI')

    #name tickers and correct time stamp

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = dates.date()+datetime.timedelta(days=1)
        
        else:
            date = dates.date()

    else:
        code = codes
        date = dates.date()
        

    try:    
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                 date,
                                                                                 date+datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    if is_Futures: #若為期貨
            ticks = api.ticks(
                contract = api.Contracts.Futures.get(code)[codes],  # For futures, use the Futures contract
                date=date.strftime('%Y-%m-%d') 
                )

    else: #若為證券
            ticks = api.ticks(
            contract=api.Contracts.Stocks[code],  # For stocks, use the Stocks contract
                date=date.strftime('%Y-%m-%d')
            )    
    df = pd.DataFrame({**ticks})

    df.ts = pd.to_datetime(df.ts)
    df['code'] = code
    df = df.set_index('ts')

    return df, False

In [37]:
df, in_db = get_ticks(connection, api, '2024/10/24', '2330')

print(in_db)
df

<SecurityType.Option: 'OPT'> fetch done
True


,close,ask_price,bid_volume,ask_volume,tick_type,bid_price,volume,code
ts,,,,,,,,
2024-10-24 09:00:02.797338,1070.0,1075.0,122,2121,1,1070.0,2748,2330
2024-10-24 09:00:02.797338,1070.0,1075.0,122,2121,1,1070.0,2748,2330
2024-10-24 09:00:02.819519,1070.0,1075.0,33,2129,2,1070.0,89,2330
2024-10-24 09:00:02.819519,1070.0,1075.0,33,2129,2,1070.0,89,2330
2024-10-24 09:00:02.836538,1070.0,1075.0,2863,2129,2,1065.0,33,2330
...,...,...,...,...,...,...,...,...
2024-10-24 13:24:58.992042,1055.0,1060.0,1864,516,2,1055.0,1,2330
2024-10-24 13:24:58.992042,1055.0,1060.0,1864,516,2,1055.0,1,2330
2024-10-24 13:24:59.478423,1060.0,1060.0,1861,516,1,1055.0,1,2330


In [38]:
def update_ticks(connection, api, daily_target, is_Futures = False):
    main_df = pd.DataFrame()

    tw_calendar = get_calendar('XTAI')

    for date, codes in daily_target.items():

        day_trading_codes = [code for code in codes ]

        print('正在更新{}逐筆成交資料，總共{}檔標的，為{}'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes),day_trading_codes))

        for code in day_trading_codes:
            df, in_db = get_ticks(connection, api, date, code, is_Futures)

            if df is not None and in_db:
                main_df = pd.concat([main_df, df], sort = False)
                time.sleep(1)

            prev_trading_date = tw_calendar.previous_close(date).date()
            prev_df, prev_in_db = get_ticks(connection, api, date, code, is_Futures)

            if prev_df is not None and not prev_in_db:
                main_df = pd.concat([main_df, prev_df], sort = False)
                time.sleep(1)
    
    if not main_df.empty:
        try:
            main_df.to_sql('ticks', connection, if_exists='append', index=False)
            print("Data stored successfully.")
        except Exception as e:
            print("Failed to store data:", e)
    return main_df



In [39]:
daily_targets ={
    pd.to_datetime('2024-08-22'): ['2317', '2330'],
    pd.to_datetime('2024-08-23'): ['2317', '2330'],
    pd.to_datetime('2024-08-24'): ['2317', '2330']
                }

In [40]:
connection = sqlite3.connect('work_data.db')
update_ticks(connection, api, daily_targets)

正在更新2024/08/22逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/23逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/24逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.


,tick_type,bid_price,bid_volume,ask_volume,volume,close,ask_price,code
ts,,,,,,,,
2024-08-22 09:00:00.362644,1.0,183.5,374.0,252.0,831.0,184.0,184.0,2317
2024-08-22 09:00:01.879958,1.0,183.5,400.0,305.0,1.0,184.0,184.0,2317
2024-08-22 09:00:01.967940,1.0,183.5,400.0,303.0,2.0,184.0,184.0,2317
2024-08-22 09:00:02.593794,1.0,183.5,401.0,302.0,1.0,184.0,184.0,2317
2024-08-22 09:00:04.008736,1.0,183.5,403.0,305.0,1.0,184.0,184.0,2317
...,...,...,...,...,...,...,...,...
2024-08-23 13:24:54.884657,1.0,949.0,62.0,102.0,1.0,950.0,950.0,2330
2024-08-23 13:24:55.499025,1.0,949.0,62.0,101.0,1.0,950.0,950.0,2330
2024-08-23 13:24:57.996492,2.0,949.0,63.0,102.0,1.0,949.0,950.0,2330


In [45]:
def retry_request(url, payloads, headers):
    
    for i in range(3):
    
        try:
            return requests.get(url, params= payloads, headers=headers)
        
        except:
            print('發生錯誤，請等待一分種後再嘗試')
            time.sleep(60)
    
    return None

def get_daily_prices(date, connection):

    try:
        df = pd.read_sql("select * from daily_prices where 日期 = '{}'".format(date),
                         connection,
                         parse_dates= ['日期'],
                         index_col= ['證券代號','日期'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    #INDB TRUE
    #先在資料庫裡抓取每日收盤價，沒有資料則利用爬蟲重新抓取
    
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'

    payloads = {
        'response': 'html',
        'date': date.strftime('%Y%m%d'),
        'type': 'ALLBUT0999'
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'
    }

    response = retry_request(url, payloads, headers)

    try:
        df = pd.read_html(response.text)[-1]
    
    except:
        print('{} 找不到資料'.format(
            date.strftime('%Y%m%d')))
        
        return None, False
    #INDB False
    df.columns = df.columns.get_level_values(2)
    df['漲跌價差'] = df['漲跌價差'].where(df['漲跌(+/-)'] != '-', -df['漲跌價差'])
    df.drop(['證券名稱','漲跌(+/-)'],inplace=True, axis=1)
    df['日期'] = pd.to_datetime(date)
    df = df.set_index(['證券代號','日期'])
    df = df.apply(pd.to_numeric, errors = 'coerce')
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
    df['股價震幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100

    return df, False



In [46]:
def update_daily_prices(start_date, end_date, connection):

    tw_calendars = get_calendar("XTAI")

    main_df = pd.DataFrame()

    for date in pd.date_range(start_date, end_date):

        if date not in tw_calendars.opens:
            continue

        df, in_db = get_daily_prices(date, connection)

        if df is not None and not in_db:
            main_df = pd.concat([main_df,df],axis = 0)
            print('{} 每日收盤行情抓取完成，等待15秒'.format(date.strftime('%Y%m%d')))
            time.sleep(15)


    if not main_df.empty:
        main_df.to_sql('daily_price', connection, if_exists='append')
        
        return main_df

    

In [49]:
def update_historical_data(start_date, end_date, daily_target, connection, api):

    tw_calendar = get_calendar('XTAI')

    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    print('正在更新每日收盤行情...')

    update_daily_prices(tw_calendar.previous_close(start).date(), end, connection)

    print('正在更新逐筆交易...')
    
    update_ticks(connection, api, daily_targets)

    print('股市資料更新完成')


In [50]:
update_historical_data('2024-10-01','2024-10-26', '2330', connection, api)

正在更新每日收盤行情...
20240930 每日收盤行情抓取完成，等待15秒
20241001 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241002 找不到資料
20241003 找不到資料
20241004 每日收盤行情抓取完成，等待15秒
20241007 每日收盤行情抓取完成，等待15秒
20241008 每日收盤行情抓取完成，等待15秒
20241009 每日收盤行情抓取完成，等待15秒
20241014 每日收盤行情抓取完成，等待15秒
20241015 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241016 每日收盤行情抓取完成，等待15秒
20241017 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241018 每日收盤行情抓取完成，等待15秒
20241021 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241022 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241023 每日收盤行情抓取完成，等待15秒
20241024 每日收盤行情抓取完成，等待15秒
發生錯誤，請等待一分種後再嘗試
20241025 每日收盤行情抓取完成，等待15秒
正在更新逐筆交易...
正在更新2024/08/22逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/23逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/24逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.
股市資料更新完成


In [51]:
ticks = {}
kbars = {}
volume_today = {}

In [53]:
def tbot(codes, is_Futures = False):
    import pandas as pd
    
    print('--- 啟動 Tbot #{} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes
    
    ticks[code] = pd.concat([get_ticks(code, prev_trading_date, connection, api)[0],get_ticks(code, date, connection, api)[0]])
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    
    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0 :

            kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
            kbars[code] = kbars[code][prev_trading_dates:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

            try:
                current_kbars = kbars[code].loc[str_current_time]
            except:
                continue

            print(current_kbars)


In [54]:
tbot('2330')

--- 啟動 Tbot #2330 ---
今日非交易日


SDK NOTICE Sat Oct 26 16:59:59.342 2024 solClient.c:11120                    (312011000) Session '(c3,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F229985348/1026/074336/868785/101.10.58.190', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 172.20.10.2:52340'


Response Code: 0 | Event Code: 12 | Info: Session '(c3,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F229985348/1026/074336/868785/101.10.58.190', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting


SDK NOTICE Sat Oct 26 17:00:02.399 2024 solClient.c:11050                    (312011000) Connect attempt for host '210.59.255.161:80' for session '(c3,s1)_sinopac(client name 'PYAPI/F229985348/1026/074336/868785/101.10.58.190', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 172.20.10.2:0')' timed out
SDK NOTICE Sat Oct 26 17:00:08.515 2024 solClient.c:11050                    (312011000) Connect attempt for host '210.59.255.161:80' for session '(c3,s1)_sinopac(client name 'PYAPI/F229985348/1026/074336/868785/101.10.58.190', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 172.20.10.2:0')' timed out
SDK NOTICE Sat Oct 26 17:00:14.553 2024 solClient.c:10769                    (312011000) Session '(c3,s1)_sinopac' error attempting transport connection, client name 'PYAPI/F229985348/1026/074336/868785/101.10.58.190', VPN name 'sinopac', peer host '21

Session Down.
Response Code: 0 | Event Code: 1 | Info: TCP connection failure for fd 94, error = Network is unreachable (51) | Event: Session down
